------------------------------------

### If any part of this notebook is used in your research, please cite with the reference found in [`README.md`](https://github.com/pysal/spaghetti#bibtex-citation).

-----------------------------

#### This notebook requires installations of:
* [`geopandas`](http://geopandas.org)
    * `$ conda install -c conda-forge geopandas`

* [`shapely`](https://shapely.readthedocs.io/en/latest/)
    * `$ conda install -c conda-forge shapely`

* [`descartes`](https://pypi.org/project/descartes/)
    * `$ conda install -c conda-forge descartes`

* [`matplotlib`](https://matplotlib.org)
    * `$ conda install matplotlib`

* [`pulp`](https://anaconda.org/conda-forge/pulp)
    * `$ pip install pulp`

* [`watermark`](https://github.com/rasbt/watermark)
    * `$ pip install watermark`

-------------

# The Traveling Sales(man)(person) Problem — TSP

#### James D. Gaboardi [<jgaboardi@gmail.com>]

-----------------

# original time tagger for update

In [4]:
import os
last_modified = None
if os.name == "posix":
    last_modified = !stat -f\
                    "# This notebook was last updated: %Sm"\
                     Traveling_Salesman_Problem.ipynb
elif os.name == "nt":
    last_modified = !for %a in (Traveling_Salesman_Problem.ipynb)\
                    do echo # This notebook was last updated: %~ta
    
if last_modified:
    get_ipython().set_next_input(last_modified[-1])

In [ ]:
# This notebook was last updated: Nov 14 08:18:30 2019

----------------

# fresh watermarks for update

In [1]:
%load_ext watermark
def notebook_watermark(packages=False):
    if not packages:
        %watermark -u -n -t -z -v
        print("")
        %watermark -m
    else:
        %watermark -iv

In [2]:
notebook_watermark()

last updated: Mon Nov 18 2019 16:39:47 EST 

CPython 3.6.7
IPython 7.9.0

compiler   : GCC Clang 9.0.0 (tags/RELEASE_900/final)
system     : Darwin
release    : 19.0.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


* even in pop culture... https://en.wikipedia.org/wiki/Travelling_Salesman_(2012_film)

https://en.wikipedia.org/wiki/Travelling_salesman_problem

formulation...

* source from Harvey Miller book
* source from Church Murray book...

* where do they get their formulations...

* cite integer programming formulation....

\begin{aligned}
\Huge \min_{w,b,\xi} \quad & \Huge \frac{1}{2}w^{t}w+C\sum_{i=1}^{N}{\xi_{i}}\\
\Huge \textrm{s.t.} \quad & \Huge y_{i}(w\phi(x_{i}+b))+\xi_{i}-1\\
\Huge & \Huge \xi\geq0    \\
\end{aligned}

### Scenario:

* newspaper boys needs to decide his route...
* his parents/(other) are renowned operations research folks...
* top woman/man in OR...
* they help newspaper boy find an optimal route...

* Traveling Salesman Problem
  * ... textual description...
  * ... mathematical programming formulation...
  * ... references ... 


---------------------

In [4]:
import spaghetti as spgh
from libpysal import examples
import pulp
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from shapely.geometry import Point, LineString

try:
    from IPython.display import set_matplotlib_formats
    set_matplotlib_formats('retina')
except ImportError:
    pass

%matplotlib inline

__author__ = "James Gaboardi <jgaboardi@gmail.com>"

In [5]:
notebook_watermark(packages=True)

matplotlib 3.1.2
spaghetti  1.3.1
geopandas  0.6.2
libpysal   4.1.1



In [9]:
pulp.VERSION

'1.6.8'

In [10]:
def _get_packages(self, pkgs):
    if self.out:
        self.out += '\n'
    packages = pkgs.split(',')

    for p in packages:
        if p == 'scikit-learn':
            p = 'sklearn'
        try:
            imported = __import__(p)
        except ImportError:
            ver = 'not installed'
        else:
            try:
                ver = pkg_resources.get_distribution(p).version
            except DistributionNotFound:
                try:
                    ver = imported.__version__
                except AttributeError:
                    try:
                        ver = imported.version
                    except AttributeError:
                        try:
                            ver = imported.version_info
                        except AttributeError:
                            ver = 'unknown'

        self.out += '\n%s %s' % (p, ver)

In [13]:
import types
import pkg_resources
from pkg_resources import DistributionNotFound

In [14]:
def _print_all_import_versions(vars):
    to_print = set()
    for val in list(vars.values()):
        if isinstance(val, types.ModuleType):
            if val.__name__ != 'builtins':
                try:
                    to_print.add((val.__name__, val.__version__))
                except AttributeError as e:
                    try:
                        imported = __import__(val.__name__.split('.')[0])
                        to_print.add((imported.__name__,
                                      imported.__version__))
                    except AttributeError as e:
                        continue

In [15]:
_print_all_import_versions(vars)

AttributeError: 'builtin_function_or_method' object has no attribute 'values'

In [16]:
import watermark

In [17]:
print(dir(watermark))

['DistributionNotFound', 'IPython', 'Magics', 'PackageNotFoundError', 'WaterMark', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'argument', 'cpu_count', 'datetime', 'gethostname', 'line_magic', 'load_ipython_extension', 'magic_arguments', 'magics_class', 'parse_argstring', 'pkg_resources', 'platform', 'strftime', 'subprocess', 'sys', 'time', 'types', 'watermark']


In [72]:
"""
IPython magic function to print date/time stamps and
various system information.

Author: Sebastian Raschka <sebastianraschka.com>
License: BSD 3 clause
"""

#from . import __version__
import platform
import subprocess
from time import strftime
from time import time
import datetime
from socket import gethostname
from multiprocessing import cpu_count
import types
import pkg_resources
from pkg_resources import DistributionNotFound

import IPython
from IPython.core.magic import Magics
from IPython.core.magic import magics_class
from IPython.core.magic import line_magic
from IPython.core.magic_arguments import argument
from IPython.core.magic_arguments import magic_arguments
from IPython.core.magic_arguments import parse_argstring


class PackageNotFoundError(Exception):
    pass


@magics_class
class James_WaterMark(Magics):
    """
    IPython magic function to print date/time stamps
    and various system information.

    """
    @magic_arguments()
    @argument('-a', '--author', type=str,
              help='prints author name')
    @argument('-d', '--date', action='store_true',
              help='prints current date as YYYY-mm-dd')
    @argument('-n', '--datename', action='store_true',
              help='prints date with abbrv. day and month names')
    @argument('-t', '--time', action='store_true',
              help='prints current time as HH-MM-SS')
    @argument('-i', '--iso8601', action='store_true',
              help='prints the combined date and time including the time zone'
                   ' in the ISO 8601 standard with UTC offset')
    @argument('-z', '--timezone', action='store_true',
              help='appends the local time zone')
    @argument('-u', '--updated', action='store_true',
              help='appends a string "Last updated: "')
    @argument('-c', '--custom_time', type=str,
              help='prints a valid strftime() string')
    @argument('-v', '--python', action='store_true',
              help='prints Python and IPython version')
    @argument('-p', '--packages', type=str,
              help='prints versions of specified Python modules and packages')
    @argument('-h', '--hostname', action='store_true',
              help='prints the host name')
    @argument('-m', '--machine', action='store_true',
              help='prints system and machine info')
    @argument('-g', '--githash', action='store_true',
              help='prints current Git commit hash')
    @argument('-r', '--gitrepo', action='store_true',
              help='prints current Git remote address')
    @argument('-b', '--gitbranch', action='store_true',
              help='prints current Git branch')
    @argument('-w', '--watermark', action='store_true',
              help='prints the current version of watermark')
    @argument('-iv', '--iversions', action='store_true',
              help='prints the name/version of all imported modules')
    @line_magic
    def james_watermark(self, line):
        """
        IPython magic function to print date/time stamps
        and various system information.
        """
        self.out = ''
        args = parse_argstring(self.watermark, line)

        if not any(vars(args).values()) or args.iso8601:
            try:
                dt = datetime.datetime.fromtimestamp(int(time()),
                                                     datetime.timezone.utc)
                iso_dt = dt.astimezone().isoformat()
            except AttributeError:  # timezone only supported by Py >=3.2:
                iso_dt = strftime('%Y-%m-%dT%H:%M:%S')

        if not any(vars(args).values()):
            self.out += iso_dt
            self._get_pyversions()
            self._get_sysinfo()

        else:
            if args.author:
                self.out += '% s ' % args.author.strip('\'"')
            if args.updated and args.author:
                self.out += '\n'
            if args.updated:
                self.out += 'last updated: '
            if args.custom_time:
                self.out += '%s ' % strftime(args.custom_time)
            if args.date:
                self.out += '%s ' % strftime('%Y-%m-%d')
            elif args.datename:
                self.out += '%s ' % strftime('%a %b %d %Y')
            if args.time:
                self.out += '%s ' % strftime('%H:%M:%S')
            if args.timezone:
                self.out += '%s ' % strftime('%Z')
            if args.iso8601:
                self.out += iso_dt
            if args.python:
                self._get_pyversions()
            if args.packages:
                self._get_packages(args.packages)
            if args.machine:
                self._get_sysinfo()
            if args.hostname:
                space = ''
                if args.machine:
                    space = '  '
                self.out += '\nhost name%s: %s' % (space, gethostname())
            if args.githash:
                self._get_commit_hash(bool(args.machine))
            if args.gitrepo:
                self._get_git_remote_origin(bool(args.machine))
            if args.gitbranch:
                self._get_git_branch(bool(args.machine))
            if args.iversions:
                self._print_all_import_versions(self.shell.user_ns)
            if args.watermark:
                if self.out:
                    self.out += '\n'
                self.out += 'watermark %s' % __version__
        print(self.out.strip())

    def _get_packages(self, pkgs):
        if self.out:
            self.out += '\n'
        packages = pkgs.split(',')

        for p in packages:
            if p == 'scikit-learn':
                p = 'sklearn'
            try:
                imported = __import__(p)
            except ImportError:
                ver = 'not installed'
            else:
                try:
                    ver = pkg_resources.get_distribution(p).version
                except DistributionNotFound:
                    try:
                        ver = imported.__version__
                    except AttributeError:
                        try:
                            ver = imported.version
                        except AttributeError:
                            try:
                                ver = imported.version_info
                            except AttributeError:
                                ver = 'unknown'

            self.out += '\n%s %s' % (p, ver)

    def _get_pyversions(self):
        if self.out:
            self.out += '\n\n'
        self.out += '%s %s\nIPython %s' % (
            platform.python_implementation(),
            platform.python_version(),
            IPython.__version__)

    def _get_sysinfo(self):
        if self.out:
            self.out += '\n\n'
        self.out += ('compiler   : %s\nsystem     : %s\n'
                     'release    : %s\nmachine    : %s\n'
                     'processor  : %s\nCPU cores  : %s\ninterpreter: %s') % (
            platform.python_compiler(),
            platform.system(),
            platform.release(),
            platform.machine(),
            platform.processor(),
            cpu_count(),
            platform.architecture()[0])

    def _get_commit_hash(self, machine):
        process = subprocess.Popen(['git', 'rev-parse', 'HEAD'],
                                   shell=False,
                                   stdout=subprocess.PIPE)
        git_head_hash = process.communicate()[0].strip()
        space = ''
        if machine:
            space = '   '
        self.out += '\nGit hash%s: %s' % (space,
                                          git_head_hash.decode("utf-8"))

    def _get_git_remote_origin(self, machine):
        process = subprocess.Popen(['git', 'config', '--get',
                                    'remote.origin.url'],
                                   shell=False,
                                   stdout=subprocess.PIPE)
        git_remote_origin = process.communicate()[0].strip()
        space = ''
        if machine:
            space = '   '
        self.out += '\nGit repo%s: %s' % (space,
                                          git_remote_origin.decode("utf-8"))

    def _get_git_branch(self, machine):
        process = subprocess.Popen(['git', 'rev-parse', '--abbrev-ref',
                                    'HEAD'],
                                   shell=False,
                                   stdout=subprocess.PIPE)
        git_branch = process.communicate()[0].strip()
        space = ''
        if machine:
            space = ' '
        self.out += '\nGit branch%s: %s' % (space,
                                            git_branch.decode("utf-8"))

    @staticmethod
    def _print_all_import_versions(vars):
        to_print = set()
        for val in list(vars.values()):
            if isinstance(val, types.ModuleType):
                if val.__name__ != 'builtins':
                    try:
                        to_print.add((val.__name__, val.__version__))
                    except AttributeError as e:
                        try:
                            imported = __import__(val.__name__.split('.')[0])
                            to_print.add((imported.__name__,
                                          imported.__version__))
                        except AttributeError as e:
                            continue

        longest = max([len(i[0]) for i in to_print] + [0]) + 1
        for entry in to_print:
            print(('%s' % entry[0]).ljust(longest) + entry[1])


def load_ipython_extension(ipython):
    ipython.register_magics(James_WaterMark)

In [73]:
wm = James_WaterMark()

In [74]:
%load_ext james_watermark

ModuleNotFoundError: No module named 'james_watermark'

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


# --> get headings from Facility Location notebook.

### Streets as empirical data

In [8]:
streets = gpd.read_file(examples.get_path('streets.shp'))
streets.crs = {'init':'epsg:2223'}
streets = streets.to_crs(epsg=2762)
streets.head()

,ID,Length,geometry
0,1.0,244.116229,"LINESTRING (222006.581 267347.973, 222006.609 ..."
1,2.0,375.974828,"LINESTRING (222006.401 267549.141, 222006.581 ..."
2,3.0,400.353405,"LINESTRING (221419.878 267804.150, 221410.853 ..."
3,4.0,660.000000,"LINESTRING (220874.568 268352.647, 220803.400 ..."
4,5.0,660.000000,"LINESTRING (220801.878 268398.082, 220916.451 ..."


----------------------